<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px"> 
# Lesson: PyMC & Bayesian Regression

_Authors: Tim Book, Matt Brems, Noelle Brown_

## LEARNING OBJECTIVES
By the end of the lesson, students should be able to:
- Define conjugacy.
- Fit models in PyMC.
- Interpret traceplots, posterior distributions, and posterior predictions.

---

## Recap
Remember that parameters have distributions. In Bayesian statistics, our goal is to **find the posterior distribution of our parameter of interest**.

<details><summary>In order to generate a posterior distribution, what are the two components we need?</summary>

- Prior distribution summarizing our beliefs about the parameter before observing any data.
- Likelihood function summarizing how our data were generated.
</details>

## Why Bayes?
There are several reasons why we would prefer to use a Bayesian technique over a frequentist one. What are they? (THREAD)

<details>
    <summary>What are some reasons to use Bayesian techniques?</summary>
    
* When you have prior knowledge of a situation (ie, you might no better than data suggests)
* When you have small data that can benefit from additional subject matter expertise
* When you want to be able to make advanced probabilistic statements that are unavailable to frequentist techniques
</details>

# Markov Chain Monte Carlo Methods

Monte Carlo simulations are used to model much more complicated things as well. In calculus, if you remember calculating the integral (area under the curve), you had to remember those nasty "integration by parts" formulas and "u-substitution." Monte Carlo simulations make that [relatively trivial with computers](https://www.youtube.com/watch?v=GZXhGwCXct0)!

You may remember that we discussed Monte Carlo simulations all the way back in the Introduction to Probability lesson. Some of the problems in the notebook included "Suppose you roll three dice. What is the probability that the three dice are rolled in increasing order?" and "I flip my coin until I flip heads. I count up the number of coins I flipped and roll that many dice. What is the probability that the average roll will be between 3 and 4 (inclusive)?"
- These might be good probability problems to practice!

Monte Carlo simulations aren't inherently Bayesian by themselves. But the way that `PyMC3` and most programs will find the posterior distribution of the parameter of interest will be to use Markov Chain Monte Carlo methods, or MCMC methods.

## Primer on Markov Chain Monte Carlo Methods

Here's a quick, three-minute run-down of Markov Chain Monte Carlo methods.

There are three main components to MCMC Methods.
- Monte Carlo Methods
- Markov Chains
- Acceptance-Rejection Sampling

If we were to just generate random numbers without any rhyme or reason, we'd be doing basic Monte Carlo simulations.
<img src="./images/montecarlo.gif" alt="drawing" width="500"/>

Suppose that, instead, we're still generating random numbers. However, this time, the last random number we generate influences the next random number we generate. This incorporates Markov chains.
<img src="./images/markovchain.gif" alt="drawing" width="600"/>

The final component, "acceptance-rejection sampling," will take each random sample and accept it or reject it, based on how "good" it is. The algorithm for doing this uses the prior and likelihood that you came up with to determine if this sample is "good enough." The basic algorithm is called the [Metropolis-Hastings Algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).

<img src="./images/acceptancerejection.gif" alt="drawing" width="550"/>

All of this will happen when you run `pymc3.sample()` below!

---

### Acceptance-Rejection Sampling

- Acceptance-rejection sampling is a specific type of Monte Carlo sampling.

- We’re going to sample some observation, then decide whether to keep it (accept it) or discard it (reject it). 
    - Specifically, we will accept these values a certain percentage of the time and reject a certain percentage of the time.

- As our random walk moves about, we need some way to make sure that we’re moving in the right direction.
    - If we are moving in the right direction, we’ll accept that sample.
    - If we aren’t moving in the right direction, we may reject that sample.

We’ll walk through a specific algorithm to accept/reject, called the [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm).
- PyMC by default will implement a more sophisticated algorithm called the [No U-Turn Sampler](https://arxiv.org/abs/1111.4246).

The Metropolis-Hastings algorithm allows us to, at each step, identify whether we are getting “hotter” or “colder.”

#### Metropolis-Hastings Algorithm
Part 1. Generate $\theta_{proposal} \sim N(\theta_{t-1}, \sigma)$.

Part 2. Calculate the ratio of posterior probabilities $r = \frac{P(\theta_{proposal}|y)}{P(\theta_{t-1}|y)}$.
- Since we have specified the prior and the likelihood, we can just find the posterior probability of these specific values!
- If $r \ge 1$, then $\theta_{proposal}$ is a likelier value for our posterior distribution than $\theta_{t-1}$. You can think of this as "we're heading in the right direction."
- If $r < 1$, then $\theta_{proposal}$ is not a likelier value for our posterior distribution than $\theta_{t-1}$. You can think of this as "we're heading in a direction of lower probability."

Part 3. Calculate our acceptance probability $\alpha = \min\{r,1\}$.
- This is the probability that we'll accept $\theta_{proposal}$ as our new value $\theta_t$.
- If $r \ge 1$, then $\theta_{proposal}$ is a likelier value for our posterior distribution than $\theta_{t-1}$ and our acceptance probability is 100%!
- If $r < 1$, then $\theta_{proposal}$ is not a likelier value for our posterior distribution than $\theta_{t-1}$, so we may reject $\theta_{proposal}$.

Part 4. We want to get our computer to basically flip a coin with probability of success $\alpha$ to determine whether or not we accept $\theta_{proposal}$. We generate $U_t \sim Uniform(0,1)$.
- If $U_t \le \alpha$, then accept $\theta_{proposal}$. (That is, $\theta_t = \theta_{proposal}$.)
- If $U_t > \alpha$, then reject $\theta_{proposal}$. (That is, $\theta_t = \theta_{t-1}$.)

Remember that we've already seen how Markov chains and Monte Carlo simulations can work together. Let's add the acceptance/rejection component to this!


We will repeat this process a large number of times - the number of samples we generate with `pm.sample()`.

Once we do this long enough and visually inspect traceplots to convince ourselves that we have “converged” to the posterior distribution of interest, we usually discard early samples (before we converged), then we store some large $n$ of the later samples.
- We might run this multiple times (by setting `chains` > 1 in `PyMC`) to protect against converging to a local optimum.

Once we have our sample of size $n$, we can conduct whatever inference we want on the posterior distribution of the parameter of interest.
- Find the mean.
- Find the median.
- Find the 95% ‘highest posterior density.’
- Find the variance.

In [ ]:
# Install pymc3
# !pip install pymc3

In [ ]:
# Import the necessary packages:

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

import pymc3 as pm

## Example 1: Skittles!
In a snack-size pack of Skittles, you are concerned with getting an unfair amount of green Skittles (the worst flavor). You think there are about 20 skittles per pack. There are 5 different colors of Skittles. 

<img src="./images/skittles.jpeg" style="height: 300px"> 

How can we determine if we get an unfair amount of green Skittles?

**A frequentist would...**
- Open many snack-size packs of Skittles, and estimate $\hat{p}$, the probability of getting a green skittle.
- Conduct a hypothesis test to determine if $p = 0.2$.

**A Bayesian would...**
- Begin with a preconceived notion about $p$ (a prior). This prior reflects not just our belief, but the strength of our belief.
- Open many snack-size packs of Skittles and record the results (the likelihood).
- Obtain a posterior distribution for $p$ that combines our priors with our data.
- Make statements about this posterior.

What is the (posterior) distribution of the number of green skittles I might find in a pack?

In [ ]:
# Import green skittle count data



### Step 1: Define our likelihood and priors

We'll start with our priors. 

What do we know/believe about the percentage of green skittles we might find in a pack? 

<details><summary>What distribution will be best for this?</summary>

- The percentage of green skittles we might find in a pack is a number between 0 and 1.
- The Beta distribution will be good here!
</details>

#### The Beta Distribution
Recall that the beta distribution is a **continuous** distribution. It's often used in Bayesian statistics because it's one of the only continuous distributions that take values between 0 and 1. This makes it particularly convenient to use as a prior on probabilities.

You can take a closer look at the Beta distribution [here](https://en.wikipedia.org/wiki/Beta_distribution)!

If $X \sim \text{Beta}(a, b)$, then

* $\text{E}[X] = \frac{a}{a + b}$
* $\text{Var}[X] = \frac{ab}{(a + b)^2(a + b + 1)}$

Also note: The $\text{Beta}(1, 1)$ distribution is actually the $\text{Uniform}(0, 1)$ distrubtion!

Let's compute some beta means and variances. What do you think is a reasonable prior for our problem?

<details><summary>In my opinion, setting up your priors and likelihoods is the hardest part of Bayesian inference.</summary>

![](./images/modified_bayes.png)

</details>

In [ ]:
def beta_stats(a, b):
    mean_ = a / (a + b)
    var_ = a*b / ((a + b)**2 * (a + b + 1))
    return mean_, var_

In [ ]:
# We want the mean to be about 0.2 (20% of each flavor)
beta_stats( , )

In [ ]:
# Prior for percentage of green skittles we might find in a pack



<details><summary>We can also set a prior on the number of Skittles that are in a pack! What distribution will be best for this?</summary>

- The number of Skittles in each pack is count data.
- The Poisson distribution will work well here!
</details>

In [ ]:
# Prior for number of Skittles in a pack



Now, let's define our likelihood. 

We are interested in seeing how many green Skittles are in a bag.

There's a fixed number of Skittles in a bag. Each of those Skittles has some probability of being green or not green.
We are looking to see if a Skittle is green or not in our bag. 

A good tip from [Bayesian Methods for Hackers:](https://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter2_MorePyMC/Ch2_MorePyMC_PyMC3.ipynb)  
> "A good starting thought to Bayesian modeling is to think about how your data might have been generated. Position yourself in an omniscient position, and try to imagine how you would recreate the dataset."

<details><summary>What distribution will be best for this?</summary>

- Each Skittle has some probability of being green and some probability of being not green.
- The Binomial distribution will work here.
</details>

In [ ]:
# Likelihood



### MCMC Using PyMC3
Markov chain Monte Carlo methods (commonly abbreviated MCMC methods) made conjugacy mostly unnecessary.

![](./images/pymc3-res.png)

[TL;DR](https://en.wikipedia.org/wiki/TL;DR) of MCMC: We use computers to take our prior and likelihood, then simulate a large number of samples from the posterior.
- If I want to find the mean of my posterior distribution, then I can just do `np.mean(xs)`, where `xs` is my saved list of samples from my posterior.
- If I want to find the median of my posterior distribution, then I can do `np.median(xs)`.
- If I want to find the 95% [credible interval](http://www.statisticshowto.com/credible-interval/) for my parameter based on the posterior distribution of my parameter, then I can just find the 2.5th and 97.5th percentiles of `xs`!

Today, we're going to use `PyMC3`, a Bayesian modeling library in Python that will enable us to use Markov chain Monte Carlo methods. Luckily, this is a package that is geared toward scientists, not necessarily statisticians. We'll be able to use what we currently know about Bayesian statistics to leverage this *really* powerful library.

### The Situation:
$$
\begin{align}
    X | p &\sim \text{Binomial}(n, p) \\
        p &\sim \text{Beta}(a, b)
\end{align}
$$

We've also observed a certain number of greens in our 20 bags (our data).

### PyMC3 models live in "contexts" instead of "objects"
That means they'll look something like this:

```python
with pm.Model() as model:
    # Define priors
    # Define likelihood
    # Execute MCMC
```

### Step 2: Let's set this up in PyMC3!

In [ ]:
# Use PyMC3 to solve for the posterior distribution



In [ ]:
# Visualize the output


In [ ]:
# Expected mean that an individual skittle will be grean


In [ ]:
# expected amount of green skittles is number of skittles in bag * prob of green
mean_green = 

In [ ]:
# What's the probability we will get an expected value of between 5 and 6 green skittles?


## Example 2: Election Day

Let's review the election example from the other day using polling data from [FiveThirtyEight](https://projects.fivethirtyeight.com/2020-election-forecast/). This time, we will look at the probability that Trump wins.

Reminder: our polls gave us the following totals:

```python
n_biden = 7_616 # number of successes
n_surveys = 14_995 # number of trials
```

In [ ]:
# Here are the results of our polls.
# 1 = Trump led the poll
# 0 = Biden led the poll
polls = np.array([1]*7379 + [0]*7616)

### Step 1: Establish our priors and our likelihood

Here we are looking for the percentage of votes Trump is likely to win in the election.

An important thing to note is that there is no such thing as sample data for problems like this, since elections only ever happen once. This is why pollsters conduct polls - to gather information about what will _probably_ happen in the future. Polls aren't perfect though, and they also change over time. **We know through personal experience that most elections end up being very close to 50/50.**

We can use the Beta distribution for our prior. We need a mean of .5 (because we expect the election to be 50/50).

_Hint:_ The priors are as subjective as they can get here.

In [ ]:
# Prior


In [ ]:
# Likelihood


### Step 2: Set up the PyMC3 model!

In [ ]:
# PyMC3 model


In [ ]:
# Traceplot


In [ ]:
# Expected value


In [ ]:
# Expected value that Trump will get more than 50% of the votes


Remember that we didn't take any outside factors into account (which FiveThirtyEight does) or even talk about the electoral college. Read more about FiveThirtyEight's modeling [here](https://fivethirtyeight.com/features/how-fivethirtyeights-2020-presidential-forecast-works-and-whats-different-because-of-covid-19/).

## Example 3: Bayesian Regression for Pay Equity
It is an often-recited statistic that women are paid about 77 cents per dollar a man makes for equal work. But how is this number (and others like it) calculated? We actually conduct a log-regression based on a person's legitimate skill level (ie, things such as education or years of experience that may legitimately contribute to you being paid more) and an indicator for the protected class. For example:

$$ \log W = \beta_0 + \beta_1\text{skill} + \beta_2\text{class} $$

After exponentiating both sides:

$$ W = e^{\beta_0 + \beta_1\text{skill}}e^{\beta_2\text{class}} $$

Where $e^{\beta_2\text{class}}$ simplifies to:

* $e^{\beta_2}$ if you are the (potentially) discriminated class.
* $e^0 = 1$ if you are not the (potentially) discriminated class.

Thus, given a dataset of salaries at a company, we might seek to estimate $e^{\beta_2\text{class}}$ to prove/disprove pay equity discrimination. Using the power of Bayesian statistics, we can actually begin with a prior assumption of _no_ pay inequity in order to allow the data to contradict it if possible.

In [ ]:
# Import wages data



### Step 1: What are the priors and likelihood?
We actually have _four_ variables to set priors on here. Most of them we actually don't have any relevant prior information for. We might set these as **noninformative priors** - that is, priors that are incredibly vague and provide no information:

* $\beta_0$ and $\beta_1$ - noninformative, we have no prior information.
* $\sigma$ - the model's variance - also noninformative, we have no good guesses.
* $\beta_2$ - the coefficient of protected class. We'll put a prior that assumes $\beta_2 \approx 0$ in order to assume innocence.

For the likelihood, remember our parametrization of OLS as a GLM:

$$Y = \mathbf{x}^T\beta + \varepsilon$$

where $\varepsilon \sim N(0, \sigma)$, and so

$$ Y \sim N(\mathbf{x}^T\beta, \sigma) $$

So we can write 

$$ \log W \sim N(\beta_0 + \beta_1\text{skill} + \beta_2\text{class}, \sigma) $$

In [ ]:
# Priors


# Likelihoods



### Step 2: PyMC3 Setup

In [ ]:
# PyMC3 Model



In [ ]:
# Traceplot

In [ ]:
# exponentiate beta_2

In [ ]:
# expected value for e^beta_2

In [ ]:
# Interpret this

### Compare to `statsmodels`.

In [ ]:
import statsmodels.api as sm

X = wages.loc[:, ["skill", "class"]]
X = sm.add_constant(X)
y = np.log(wages["wage"])
lm = sm.OLS(y,X).fit()
lm.summary()

In [ ]:
# This was actually simulated to be 0.8
np.exp(-0.1853)